# Quickstart

This quickstart shows how enact can be used to implement a ChatGPT-style
application, which can rewind the conversation to an earlier state and explore
different conversation paths.

## Prerequisites

This notebook requires an OpenAI API key. The notebook will look for the key
in the environment variable `OPENAI_API_KEY` and the file `~/openai.key`.

In [1]:
%pip install -q enact
%pip install -q openai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os

api_key: str = None

if 'OPENAI_API_KEY' in os.environ:
  print('Using OpenAI API key from environment variable.')
  api_key = os.environ['OPENAI_API_KEY']
else:
  path = os.path.join(os.path.expanduser('~'), 'openai.key')
  print(f'Checking {path} for api key')
  if os.path.exists(path):
    print(f'Found api key at {path}')
    api_key = open(path).read().strip()

assert api_key, 'Please provide API key.'


Checking /home/leo/openai.key for api key
Found api key at /home/leo/openai.key


## Assistant Chat

In [3]:
import enact
import openai  # type: ignore

openai.api_key = api_key

We will first define two enact resources to represent messages
and conversations:

In [4]:
from typing import List
import dataclasses

@enact.register
@dataclasses.dataclass
class Message(enact.Resource):
  role: str  # "system", "assistant" or "user"
  content: str

@enact.register
@dataclasses.dataclass
class Conversation(enact.Resource):
  messages: List[Message]

Next, we define an `Invokable` resource that calls the GPT API.

In [5]:
@enact.typed_invokable(
  input_type=Conversation,
  output_type=Message)
class GPT(enact.Invokable[Conversation, Message]):
  def call(self, conversation: Conversation) -> Message:
    response = openai.ChatCompletion.create(
      model='gpt-3.5-turbo',
      messages=[c.to_resource_dict() for c in conversation.messages])
    return Message.from_fields(response['choices'][0]['message'])

gpt = GPT()
gpt(Conversation(messages=[Message(role='user', content='Hello!')]))

Message(role='assistant', content='Hi there! How can I assist you today?')

Next, we define a 'program' that represents the conversation between a user and
an assistant. This is just a loop that alternates between querying the user and
GPT.

The invokable takes no input (hence `enact.NoneResource`) and returns a
conversation when a user types 'exit'.

In [6]:
@enact.typed_invokable(type(None), Conversation)
@dataclasses.dataclass
class UserConversation(
    enact.Invokable[type(None), Conversation]):
  """A conversation with a user."""
  assistant: enact.Invokable[Conversation, Message] = dataclasses.field(
    default_factory=GPT)

  def call(self) -> Conversation:
    conversation = Conversation(messages=[])
    # A conversation is a loop of user messages and assistant messages.
    while True:
      # Sample user message.
      user_content = enact.request_input(
        requested_type=str, for_value=conversation)
      if user_content == 'exit':
        return conversation

      # Append the user message to the conversation.
      conversation.messages.append(
        Message(role='user', content=str(user_content)))
      
      # Call the assistant.
      assistant_message = self.assistant(conversation)
      conversation.messages.append(assistant_message)
      

In order to resolve user inputs, enact needs to run a managed execution, called
an `Invocation`. Invocations are journaled, and need to be performed in the 
context of an enact `Store` object, which dictates where the generated resources
are persisted.

When user input is required, enact raises an `InputRequest` exception, and the
invocation needs to be continued once it is resolved. The `InvocationGenerator`
provides a convenient python generator interface to step through multiple
input requests until the execution is complete.

To sample user input we will use the python `input` function. This will open
an input window in the jupyter notebook environment.

In [7]:
import sys

store = enact.FileStore(root_dir='store_data')

def prompt_user(messages):
  """Displays the previous message and prompts the user."""
  for message in messages[-2:]:
    print(f'{message.role}: {message.content}')
    sys.stdout.flush()
  return input()

with store:
  invocation_generator = enact.InvocationGenerator(UserConversation())
  for input_request in invocation_generator:
    user_input = prompt_user(input_request.for_value().messages)
    invocation_generator.set_input(user_input)

user: Do you think "enact" is a good name for a generative software framework?
assistant: Yes, "enact" could be a good name for a generative software framework. It conveys a sense of action, implementation, and bringing ideas to life. It suggests that the framework can be used to enact or create various things. However, the final judgment of a name depends on factors such as brand positioning, target audience, and competition. It's important to consider those aspects and ensure the name aligns with the framework's purpose and goals.
user: What else could I call it?
assistant: Here are a few alternative name suggestions for a generative software framework:

1. "EvoGen" (short for Evolutionary Generator)
2. "SynthiCreate" (combining synthesis and creation)
3. "InfiniForge" (implies infinite possibilities and forging creation)
4. "GeniSys" (a play on "genius" and "system" to signify intelligent generation)
5. "VivoCraft" (a combination of "vivo" meaning life and "craft" signifying creatio

The invocation is now complete. The full execution record of our
`UserConversation` invokable is stored in the invocation object. To
illustrate, let's look at the details of one of the tracked calls
that happened during execution:

In [8]:
with store:
  invocation = invocation_generator.invocation
  enact.pprint(invocation.get_child(1), skip_repeated_refs=True)

Invocation:
  request:
    -> Request#8865e1:
      invokable: -> GPT()#2e772c
      input: -> Conversation#b60b3f: messages: [ Message(role='user', content='Do you think "enact" is a good name for a generative software framework?')]
  response:
    -> Response#76d775:
      invokable: -> GPT#2e772c
      output: -> Message(role='assistant', content='Yes, "enact" could be a good name for a generative software framework. It conveys a sense of action, implementation, and bringing ideas to life. It suggests that the framework can be used to enact or create various things. However, the final judgment of a name depends on factors such as brand positioning, target audience, and competition. It\'s important to consider those aspects and ensure the name aligns with the framework\'s purpose and goals.')#df1418
      raised: None
      raised_here: False
      children: []


We can see all execution details, including the specific invokable that was
called, the input it received and the output it generated.

## Rewinding the state of the conversation

Invocations can additionally be rewound and replayed.

For example, to regenerate the last answer by GPT and continue the conversation
from there, we can simply `rewind` the invocation.

In [10]:
with store:
  earlier_state = invocation.rewind(2)  # Undo user 'exit' and GPT response.
  invocation_generator = enact.InvocationGenerator(
    from_invocation=earlier_state)
  for input_request in invocation_generator:
    user_input = prompt_user(input_request.for_value().messages)
    invocation_generator.set_input(user_input)

user: What else could I call it?
assistant: Here are some alternative suggestions for naming a generative software framework:

1. "GenerateX"
2. "Genesis"
3. "CreateVue"
4. "CreaGen"
5. "InnovatePro"
6. "GeneraSys"
7. "IdeaForge"
8. "ArtiCraft"
9. "CodeSynth"
10. "ProtoSpark"

Remember to consider factors such as uniqueness, relevance, and brand positioning while choosing a name. It's also a good idea to check for existing software frameworks or projects with similar names to avoid confusion and trademark issues.
user: Those are terrible names!
assistant: I apologize if the suggested names did not meet your expectations. Coming up with a perfect name can be subjective and challenging. To provide more suitable alternatives, it would be helpful to understand the specific qualities or attributes you would like the name to convey related to your generative software framework. Could you provide any specific keywords or concepts you would like the name to incorporate?


We can now write a simple text interface that allows us to use the 'rewind'
command to return to an earlier conversation state:

In [11]:
with store:
  user_input = ''
  invocation_generator = enact.InvocationGenerator(UserConversation())
  while user_input != 'exit':
    for input_request in invocation_generator:
      user_input = prompt_user(input_request.for_value().messages)
      if user_input == 'rewind':
        print('\nREWINDING... *whirr*')
        invocation = invocation_generator.invocation
        invocation = invocation.rewind(3)
        invocation_generator = enact.InvocationGenerator(
          from_invocation=invocation)
        break
      invocation_generator.set_input(user_input) 

user: Ok, let's play twenty questions. Is it an animal?
assistant: Yes, it is an animal.
user: Is it larger than a cat?
assistant: Yes, it is larger than a cat.
user: Is it larger than a horse?
assistant: No, it is not larger than a horse.
user: Is it a domesticated animal?
assistant: Yes, it is a domesticated animal.
user: Ok, I give up. Just tell me what animal you thought of
assistant: I was thinking of a dog.

REWINDING... *whirr*
user: Is it a domesticated animal?
assistant: Yes, it is a domesticated animal.
user: Is it a dog?
assistant: Yes, it is a dog. Well done!
